# Flowtuple analysis demo for AIMS 2025

**Author(s):** M. Gao (magao@ucsd.edu) Ricky Mok (cskpmok@caida.org)

## 0 - Imports

In [1]:
# Import Libraries
import pyspark
from pyspark import SQLContext
from pyspark import SparkContext
import pyspark.sql.functions as psf
import pyspark.sql.types as pst
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
from pyspark.sql import SparkSession

import os
from datetime import datetime, timedelta, date
import dateutil.relativedelta as relativedelta
import IPython
import pandas as pd
import numpy as np

# Find Spark
import findspark
findspark.init()

# PySpark imports
import pyspark
from pyspark import SQLContext
from pyspark import SparkContext
from pyspark import StorageLevel
import pyspark.sql.functions as F
import pyspark.sql.types as pst
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, lit

In [2]:
print(os.environ["SPARK_HOME"])

/expanse/lustre/scratch/magao/temp_project/spark-3.2.1


## 1 - Spark Configuration

In [3]:
user=%env USER
user

'magao'

In [4]:
hostname = !hostname --fqdn
hostname[0]

'exp-9-55.expanse.sdsc.edu'

In [ ]:
PROJECT = "csd939"

In [5]:
# os.environ["PYSPARK_PYTHON"] = "/opt/anaconda3/envs/spark3/bin/python"
# os.environ["SPARK_HOME"] = "/usr/local/spark-3.1.2-bin-hadoop3.2"
os.environ["HADOOP_OPTS"] = "-Djava.library.path=/cm/shared/apps/spack/cpu/opt/spack/linux-centos8-zen/gcc-8.3.1/hadoop/3.2.2/lib/native"
os.environ['PYSPARK_SUBMIT_ARGS'] = fr"--jars /expanse/lustre/scratch/{user}/temp_project/spark-3.2.1/ext_jars/* --packages org.apache.spark:spark-avro_2.12:3.2.1 pyspark-shell"
os.environ["SPARK_LIB"] = os.environ["SPARK_HOME"]

In [ ]:
# Set your swift credentials
UCSD_NT_S3_ACCESS_KEY = ""
UCSD_NT_S3_SECRET_KEY = ""

In [ ]:
# Set spark configurations to align with the slrum configuration
SPARK_CORES_MAX = "8"
SPARK_EXEC_CPU = "2"
SPARK_EXEC_MEM = "2G"

In [6]:

MASTER_URL=fr"spark://{hostname[0]}:7077"

spark = SparkSession.builder.master(MASTER_URL).appName('spark-cluster'
).config("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider"
).config("fs.s3a.access.key", UCSD_NT_S3_ACCESS_KEY
).config("fs.s3a.secret.key", UCSD_NT_S3_SECRET_KEY
).config("fs.s3a.endpoint", "https://hermes.caida.org").config("fs.s3a.path.style.access", "true"
).config("fs.s3a.block.size", "64M").config("fs.s3a.readahead.range", "128K"
).config("fs.s3a.experimental.input.fadvise", "sequential"
).config("fs.s3a.connection.maximum", 256
).config("spark.driver.extraJavaOptions", f"-Djava.io.tmpdir=/expanse/lustre/projects/{PROJECT}/{user}/spark_scratch"
).config("spark.executor.extraJavaOptions", f"-Djava.io.tmpdir=/expanse/lustre/projects/{PROJECT}/{user}/spark_scratch"         
).config("spark.cores.max", SPARK_CORES_MAX
).config("spark.driver.cores","6"
).config("spark.driver.memory","6G"
).config("spark.executor.cores", SPARK_EXEC_CPU
).config("spark.executor.memory", SPARK_EXEC_MEM
# ).config("spark.executor.memoryOverhead", "4G"
).config("io.file.buffer.size", "67108864"
).config("spark.submit.deploymode", "client"
).config("spark.buffer.size", "67108864"
).config("spark.network.timeout", "300s"
).config("spark.sql.session.timeZone", "UTC"
).config("spark.sql.files.ignoreCorruptFiles", "true" # Set this when analyzing periods of high traffic volatility, during which there may be corrupt files
).getOrCreate()

print(spark)

:: loading settings :: url = jar:file:/expanse/lustre/scratch/magao/temp_project/spark-3.2.1/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/magao/.ivy2/cache
The jars for the packages stored in: /home/magao/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-73585ea3-401a-4b3d-8cef-ebb239439558;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.2.1 in central
	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 672ms :: artifacts dl 6ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.2.1 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	org.tukaani#xz;1.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |  

In [7]:
# sc = SparkContext(conf=spark)
# sc.setLogLevel('ERROR')
sqlcontext = SQLContext(spark)

/home/magao/miniconda3/envs/tele_ts/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [9]:
spark

## 2 - Functions

In [10]:
import socket, struct
import ipaddress

def long2ip(long):
    return ipaddress.ip_network(socket.inet_ntoa(struct.pack('!L', long)) + '/255.255.255.255', strict=False)

def long2ip24subnet(long):
    return ipaddress.ip_network(socket.inet_ntoa(struct.pack('!L', long)) + '/255.255.255.0', strict=False)

def long2ip16subnet(long):
    return ipaddress.ip_network(socket.inet_ntoa(struct.pack('!L', long)) + '/255.255.0.0', strict=False)

'''
Convert an IP string to long
'''
def ip2long(ip):
    packedIP = socket.inet_aton(ip)
    return struct.unpack("!L", packedIP)[0]

In [11]:
'''
Input: List of IPv4 Subnets in CIDR Notation e.g. ('0.0.0.0/16')

Returns the set of all IP addresses as integers within the subnet.
'''
def explode_prefix(prefix_list):
    prefix_set = set()
    
    for pfx in prefix_list:
        for ip in ipaddress.IPv4Network(pfx):
            prefix_set.add(ip2long(str(ip)))
            
    return prefix_set

In [12]:
import pytz

def load_ft_pyspark(avro_files):
    df = sqlcontext.read.format('avro').load(avro_files)
    df = df.withColumn('time', psf.to_utc_timestamp((df.time).cast(dataType=pst.TimestampType()), 'UTC'))
    # df = df.withColumn('time', psf.from_unixtime(df.time))
    return df

## 3 - Analysis Configuration

In [13]:
### Papermill Parameters ###

OUT_DIR = '/expanse/lustre/projects/{PROJECT}/{user}/shared_data/case_study_1'
START_1 = '2024-01-01 00:00'
END_1 = '2024-01-01 00:30'

## 4 - Load .avro Files

### 4.1 - Convert Timestamps to DateTime

In [14]:
# Inclusive start and end times
start_1 = pd.to_datetime(START_1)
end_1 = pd.to_datetime(END_1)

### 4.2 - Load Files

In [15]:
def generate_avro_uris(avro_filename, start_dt, end_dt, s3_pre_uri='s3a://telescope-ucsdnt-avro-flowtuple-v4-2024'):
    avro_df = pd.read_parquet(avro_filename)
    avro_df['datetime'] = pd.to_datetime(avro_df['datetime'])
    avro_df = avro_df.set_index('datetime')
    
    ### Select .avro's within timeframe
    selected_avros = avro_df[(avro_df.index >= start_dt) & (avro_df.index <= end_dt)]
    print(f'Avro filecount: {len(selected_avros)}')
    
    ### Build URI's
    selected_avro_uris = selected_avros.filename.apply(lambda x: f'{s3_pre_uri}/{x}').values
    print(f'Avro uri count: {len(selected_avro_uris)}')
    
    return selected_avro_uris

Use per-computed list of available flowtuple files. 

In [17]:
avro_filename = '../ft4_file_lists/ft4_2024_files.parquet.gzip'
s3_pre_uri = 's3a://telescope-ucsdnt-avro-flowtuple-v4-2024/'

avro_uris = generate_avro_uris(avro_filename, start_1, end_1, s3_pre_uri)

Avro filecount: 864
Avro uri count: 864


In [18]:
%%time

spark_df = load_ft_pyspark(list(avro_uris))

24/04/10 14:25:22 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


CPU times: user 35.4 ms, sys: 10.5 ms, total: 45.9 ms
Wall time: 16 s


## 5 - Optional Filters

#### Source Filters

In [19]:
src_ip_filter = set([
    # '137.110.41.27',
    # '137.110.41.248',
    # '137.110.33.50',
    # '137.110.60.41',
])

src_subnet_filter = set([
    # '137.110.0.0/16'
])

src_asn_filter = [
]

# Convert to integers
src_ip_filter = set(map(ip2long, src_ip_filter))
src_subnet_filter = explode_prefix(src_subnet_filter)
# Union
ip_filter = src_ip_filter.union(src_subnet_filter)

print(f'IP filter cardinality: {len(ip_filter)}')
print(f'ASN filter cardinality: {len(src_asn_filter)}')

IP filter cardinality: 0
ASN filter cardinality: 0


#### Dest. Filters

In [20]:
dst_ip_filter = set([
])

dst_port_filter = set([
    # 25, 1723, 2379, 3306, 4567, 5060, 6443, 7170, 7547, 8008, 8009, 8085, 8089, 9200,
    # 10250, 30005, 50001
])

print(f'DST_IP filter cardinality: {len(dst_ip_filter)}')
print(f'DST_PORT filter cardinality: {len(dst_port_filter)}')

DST_IP filter cardinality: 0
DST_PORT filter cardinality: 0


#### Misc. Filters

In [21]:
proto_filter = set([
    17
])

print(f'Protocol filter cardinality: {len(proto_filter)}')

Protocol filter cardinality: 1


### 5.1 - Apply Filters

In [22]:
def create_filter_df(items, col_name):
    return spark.createDataFrame(
        [(item,) for item in items],
        [col_name]
    )

def apply_filter_df(orig_df, filter_df, col_name):
    import pyspark.sql.functions as F
    return orig_df.join(
            F.broadcast(
                filter_df
            ),
            on=col_name
    )

In [23]:
traff_df = spark_df

if len(ip_filter) > 0:
    print(f'Applying Source IP Filter')
    ip_filter_df = create_filter_df(ip_filter, 'src_ip')
    traff_df = traff_df.join(
        F.broadcast(
            ip_filter_df
            ),
        on='src_ip'
    )

if len(src_asn_filter) > 0:
    print(f'Applying Source ASN Filter')
    asn_filter_df = create_filter_df(src_asn_filter, 'prefix2asn')
    traff_df = traff_df.join(
        F.broadcast(
            asn_filter_df
            ),
        on='prefix2asn'
    )
    
if len(dst_port_filter) > 0:
    print(f'Applying Dest. Port Filter')
    dp_filter_df = create_filter_df(dst_port_filter, 'dst_port')
    traff_df = traff_df.join(
        F.broadcast(
            dp_filter_df
            ),
        on='dst_port'
    )

if len(proto_filter) > 0:
    print(f'Applying Protocol Filter')
    proto_filter_df = create_filter_df(proto_filter, 'protocol')
    traff_df = traff_df.join(
        F.broadcast(
            proto_filter_df
            ),
        on='protocol'
    )

Applying Protocol Filter


## 6 - Traffic Metric Analysis

### Functions

In [24]:
def list_len(list):
    if list is None:
        return 0
    else:
        return len(list)

### Packet Counts + Unique Source IP Counts per Subnet

In [25]:
%%time

result = spark_df.groupby('dst_net').agg(
    F.sum('packet_cnt'), 
    F.countDistinct('src_ip')
).toPandas()

CPU times: user 1.47 s, sys: 486 ms, total: 1.96 s
Wall time: 14min 13s


In [90]:
result

,dst_net,sum(packet_cnt),count(src_ip),pc_percentile_rank,src_ip_percentile_rank
0,38,1478454591,1055898,0.701754,0.701754
1,187,1504826285,986626,0.771930,0.274854
2,4,1328710615,1057207,0.116959,0.707602
3,21,1450823828,1168539,0.526316,0.947368
4,77,1368293449,1004496,0.269006,0.426901
...,...,...,...,...,...
166,89,1368333007,984167,0.274854,0.269006
167,23,1444215824,1001236,0.461988,0.391813
168,7,1442089975,939121,0.450292,0.081871
169,88,1355154441,980680,0.175439,0.233918


In [80]:
result['dst_net'] = result['dst_net'].apply(lambda ip_int: long2ip(ip_int)).apply(lambda ip: str(ip).split('.')[1])

In [81]:
result['dst_net'] = result['dst_net'].astype(int)

In [82]:
result['pc_percentile_rank'] = result['sum(packet_cnt)'].rank(pct=True)
result['src_ip_percentile_rank'] = result['count(src_ip)'].rank(pct=True)

In [91]:
result.sort_values(by='sum(packet_cnt)', ascending=True)

,dst_net,sum(packet_cnt),count(src_ip),pc_percentile_rank,src_ip_percentile_rank
20,31,26999342,185470,0.005848,0.005848
161,134,291401414,479972,0.011696,0.011696
121,3,689296765,756486,0.017544,0.023392
55,132,725996454,728042,0.023392,0.017544
22,154,1024391745,894639,0.029240,0.046784
...,...,...,...,...,...
104,178,1888266387,1001596,0.976608,0.397661
5,44,1896421626,1983079,0.982456,0.994152
41,51,1936964962,1143055,0.988304,0.918129
170,98,1984638392,1505901,0.994152,0.982456


In [92]:
result.sort_values(by='pc_percentile_rank')

,dst_net,sum(packet_cnt),count(src_ip),pc_percentile_rank,src_ip_percentile_rank
20,31,26999342,185470,0.005848,0.005848
161,134,291401414,479972,0.011696,0.011696
121,3,689296765,756486,0.017544,0.023392
55,132,725996454,728042,0.023392,0.017544
22,154,1024391745,894639,0.029240,0.046784
...,...,...,...,...,...
104,178,1888266387,1001596,0.976608,0.397661
5,44,1896421626,1983079,0.982456,0.994152
41,51,1936964962,1143055,0.988304,0.918129
170,98,1984638392,1505901,0.994152,0.982456


In [93]:
result.sort_values(by='count(src_ip)')

,dst_net,sum(packet_cnt),count(src_ip),pc_percentile_rank,src_ip_percentile_rank
20,31,26999342,185470,0.005848,0.005848
161,134,291401414,479972,0.011696,0.011696
55,132,725996454,728042,0.023392,0.017544
121,3,689296765,756486,0.017544,0.023392
15,103,1177287039,849234,0.052632,0.029240
...,...,...,...,...,...
94,0,1420294758,1282400,0.397661,0.976608
170,98,1984638392,1505901,0.994152,0.982456
12,99,1452441648,1703488,0.543860,0.988304
5,44,1896421626,1983079,0.982456,0.994152
